In [1]:
import os
import re
import nltk
import math
import string
import numpy as np
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from natsort import natsorted
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from zipfile import ZipFile

# f = '/content/Humor,Hist,Media,Food.zip'
f = r"/content/drive/MyDrive/20_newsgroups.zip"

with ZipFile(f,'r') as zip1:
  zip1.printdir()
  zip1.extractall()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

def get_file(path,l):

  p = path
  name2 = os.listdir(p)

  doc12 = []
  label = []

  for i in range(1,len(name2)):
    # print(name[i])
    # print(i)
    
    # if '.' not in l[i]:
    #   l[i]+=".txt"
    try:
      with open(path+name2[i],encoding='utf8',errors='ignore') as f:
        lines = f.readlines()
      t1 = " ".join(lines)
      doc12.append(t1)
      label.append(l)
    except:
      df = pd.read_csv(path+name2[i],sep='\t',lineterminator='\n',names=["text"])
      df.dropna(inplace=True)
      t = df["text"].values
      # print(t)
      t1 = " ".join(t)
      doc12.append(t1)
      label.append(l)
  return doc12,label

In [6]:
doc11,label = get_file("/content/20_newsgroups/comp.graphics/","comp.graphics")
df11 = pd.DataFrame(doc11,columns=['_file_'])
df11['label'] = label
df11.head()

,_file_,label
0,Xref: cantaloupe.srv.cs.cmu.edu comp.graphics:...,comp.graphics
1,Newsgroups: comp.graphics\n Path: cantaloupe.s...,comp.graphics
2,Newsgroups: comp.graphics\n Path: cantaloupe.s...,comp.graphics
3,Xref: cantaloupe.srv.cs.cmu.edu comp.graphics:...,comp.graphics
4,Newsgroups: comp.graphics\n Path: cantaloupe.s...,comp.graphics


In [7]:
doc12,label1 = get_file("/content/20_newsgroups/sci.med/","sci.med")
df12 = pd.DataFrame(doc12,columns=['_file_'])
df12['label'] = label1

doc13,label2 = get_file("/content/20_newsgroups/talk.politics.misc/","talk.politics.misc")
df13 = pd.DataFrame(doc13,columns=['_file_'])
df13['label'] = label2

doc14,label3 = get_file("/content/20_newsgroups/rec.sport.hockey/","rec.sport.hockey")
df14 = pd.DataFrame(doc14,columns=['_file_'])
df14['label'] = label3

doc15,label4 = get_file("/content/20_newsgroups/sci.space/","sci.space")
df15 = pd.DataFrame(doc15,columns=['_file_'])
df15['label'] = label4

df = pd.concat([df11,df12,df13,df14,df15],axis=0)

In [8]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return " ".join(filtered_text)

### Preprocessing

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
nltk.download('stopwords')
# stopwords=set(stopwords.words('english'))
def cleaner(text):
    cleaned=text.replace("//"," ").replace("."," ")
    cleaned=re.sub(r'[^a-zA-Z]'," ",cleaned)
    cleaned=cleaned.strip() #removing whitespace
    cleaned=cleaned.lower() #converting into lower case words
    cleaned=re.sub(r'\w+\d+'," ",cleaned)#remove alphanumeric words
    cleaned=ps.stem(cleaned) #stemming 
    cleaned=[word for word in cleaned.split() if len(word)>2]
    cleaned=" ".join(cleaned)
    cleaned = remove_stopwords(cleaned)

    return cleaned

# df['text'] = df['text'].apply(lambda x : cleaner(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df['_file_'] = df['_file_'].apply(lambda x:cleaner(x))
df.head()

,_file_,label
0,xref cantaloupe srv cmu edu comp graphics sci ...,comp.graphics
1,newsgroups comp graphics path cantaloupe srv c...,comp.graphics
2,newsgroups comp graphics path cantaloupe srv c...,comp.graphics
3,xref cantaloupe srv cmu edu comp graphics comp...,comp.graphics
4,newsgroups comp graphics path cantaloupe srv c...,comp.graphics


Splitting the Data

In [11]:
from sklearn.model_selection import train_test_split
y=df['label']
x_train,x_test,y_train,y_test=train_test_split(df._file_,df.label,random_state=21,test_size=0.20)
res=pd.concat([x_train,y_train],axis=1)

In [ ]:
y

0      comp.graphics
1      comp.graphics
2      comp.graphics
3      comp.graphics
4      comp.graphics
           ...      
994        sci.space
995        sci.space
996        sci.space
997        sci.space
998        sci.space
Name: label, Length: 4995, dtype: object

In [ ]:
res

,_file_,label
214,newsgroups comp graphics path cantaloupe srv c...,comp.graphics
638,xref cantaloupe srv cmu edu talk abortion talk...,talk.politics.misc
133,newsgroups comp graphics path cantaloupe srv c...,comp.graphics
616,xref cantaloupe srv cmu edu comp sys mac progr...,comp.graphics
201,newsgroups rec sport hockey path cantaloupe sr...,rec.sport.hockey
...,...,...
302,xref cantaloupe srv cmu edu sci environment mi...,sci.space
710,xref cantaloupe srv cmu edu sci environment mi...,sci.space
145,xref cantaloupe srv cmu edu rec food cooking s...,sci.med
48,path cantaloupe srv cmu edu crabapple srv cmu ...,comp.graphics


In [ ]:
df_fin=res.groupby('label')['_file_'].apply(list).reset_index(name="_file")

In [ ]:
df_fin['_file']

0    [newsgroups comp graphics path cantaloupe srv ...
1    [newsgroups rec sport hockey path cantaloupe s...
2    [newsgroups sci med path cantaloupe srv cmu ed...
3    [path cantaloupe srv cmu edu magnesium club cm...
4    [xref cantaloupe srv cmu edu talk abortion tal...
Name: _file, dtype: object

In [ ]:
df

### Joing the tokens of all the documents in a class together in a list

In [ ]:
li=[]
for i in range(len(df_fin)) :
  flat = []
  j=0
  for sublist in df_fin.loc[i, "_file"]:
      if j==0:
        print(sublist.split())
      # print(sublist)
      for w in sublist.split():
        flat.append(w)
      # for item in sublist:
      #     flat.append(item)
      j+=1
  li.append(flat)
df_fin['terms'] = li
df_fin.drop(['_file'],axis=1,inplace=True)

['xref', 'cantaloupe', 'srv', 'cmu', 'edu', 'comp', 'graphics', 'comp', 'graphics', 'animation', 'newsgroups', 'comp', 'graphics', 'comp', 'graphics', 'animation', 'path', 'cantaloupe', 'srv', 'cmu', 'edu', 'das', 'news', 'harvard', 'edu', 'noc', 'near', 'net', 'howland', 'reston', 'ans', 'net', 'usc', 'utexas', 'edu', 'utnut', 'utgpu', 'utcsri', 'newsflash', 'concordia', 'cumin', 'telecom', 'uqam', 'umontreal', 'kardank', 'kardank', 'ere', 'umontreal', 'kardan', 'kaveh', 'subject', 'human', 'head', 'modeling', 'software', 'message', 'apr', 'umontreal', 'sender', 'news', 'umontreal', 'administration', 'cnews', 'organization', 'universite', 'montreal', 'references', 'apr', 'worak', 'kaist', 'wbp', 'taurus', 'nps', 'navy', 'mil', 'date', 'wed', 'apr', 'gmt', 'lines', 'article', 'wbp', 'taurus', 'nps', 'navy', 'mil', 'adaptive', 'nps', 'navy', 'mil', 'zyda', 'res', 'acct', 'writes', 'interested', 'facial', 'animation', 'want', 'implement', 'program', 'area', 'information', 'face', 'lookin

In [ ]:
df_fin

,label,terms
0,comp.graphics,"[newsgroups, comp, graphics, path, cantaloupe,..."
1,rec.sport.hockey,"[newsgroups, rec, sport, hockey, path, cantalo..."
2,sci.med,"[newsgroups, sci, med, path, cantaloupe, srv, ..."
3,sci.space,"[path, cantaloupe, srv, cmu, edu, magnesium, c..."
4,talk.politics.misc,"[xref, cantaloupe, srv, cmu, edu, talk, aborti..."


### Finding term frequency in each class

In [ ]:
temp_list=[]
temp_list2=[]
for i in range(len(df_fin)) :
  frq_list={}
  for word in df_fin.loc[i, "terms"]:
      if word not in frq_list.keys():
        frq_list[word]=1
      else:
        frq_list[word]+=1
  frq_list=dict( sorted(frq_list.items(), key=lambda x:(x[1], x[0]),reverse=True))
  temp_list2.append(frq_list)

In [ ]:
temp_list=df_fin['terms']
df_fin['attri'] = temp_list
df_fin['term_frq'] = temp_list2
df_fin.index=df_fin["label"]

In [ ]:
df_fin

,label,terms,attri,term_frq
label,,,,
comp.graphics,comp.graphics,"[xref, cantaloupe, srv, cmu, edu, comp, graphi...","[xref, cantaloupe, srv, cmu, edu, comp, graphi...","{'edu': 6078, 'graphics': 2394, 'cmu': 2080, '..."
rec.sport.hockey,rec.sport.hockey,"[newsgroups, rec, sport, hockey, path, cantalo...","[newsgroups, rec, sport, hockey, path, cantalo...","{'edu': 5775, 'cmu': 1909, 'apr': 1872, 'hocke..."
sci.med,sci.med,"[xref, cantaloupe, srv, cmu, edu, alt, comp, g...","[xref, cantaloupe, srv, cmu, edu, alt, comp, g...","{'edu': 6544, 'com': 2106, 'cmu': 2068, 'sci':..."
sci.space,sci.space,"[xref, cantaloupe, srv, cmu, edu, sci, astro, ...","[xref, cantaloupe, srv, cmu, edu, sci, astro, ...","{'edu': 6920, 'space': 2925, 'cmu': 2301, 'sci..."
talk.politics.misc,talk.politics.misc,"[xref, cantaloupe, srv, cmu, edu, talk, aborti...","[xref, cantaloupe, srv, cmu, edu, talk, aborti...","{'edu': 7336, 'com': 2946, 'cmu': 2444, 'polit..."


In [ ]:
y

0      comp.graphics
1      comp.graphics
2      comp.graphics
3      comp.graphics
4      comp.graphics
           ...      
994        sci.space
995        sci.space
996        sci.space
997        sci.space
998        sci.space
Name: label, Length: 4995, dtype: object

In [ ]:
labels=y.unique()

In [ ]:
attributes=[]
for index,row in df_fin.iterrows():
  attributes.extend(row["attri"])
attributes=np.unique(attributes)

Term Frequency

In [ ]:
tf=pd.DataFrame(columns=attributes)
for x in labels:
  for y in attributes:
    if y in df_fin.loc[x].term_frq.keys():
      tf.at[x,y]=df_fin.loc[x].term_frq[y]
tf=tf.fillna(0)

Inverse Class Frequency 

In [ ]:
icf_list=[]
for c in tf.columns:
  temp=0
  for index,row in tf.iterrows():
    if row[c]!=0:
      temp+=1
  icf_list.append(math.log(len(labels)/temp))

In [ ]:
i=0
for j in tf.columns:
  tf[j]=tf[j]*icf_list[i]
  i=i+1

In [ ]:
tf_icf=tf.to_numpy()
tf_icf_df=pd.DataFrame(tf_icf)
tf_icf_df.to_csv('file.csv')

In [ ]:
tf

,aaa,aaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaauuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuugggggggggggggggg,aaaaagggghhhh,aaaall,aaaggghhh,aaai,aaauuugggghhhhh,aacs,aad,...,zxa,zxsn,zyb,zyda,zyeh,zygon,zyue,zyxel,zzz,zzzzzz
comp.graphics,0.00000,5.180581,0.00000,0.00000,0.00000,0.00000,0.000000,2.59029,0.00000,10.361162,...,0.000000,0.00000,2.59029,5.037532,82.889293,0.000000,0.000000,10.361162,0.00000,0.00000
sci.med,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,7.770871,0.00000,0.00000,0.000000,...,0.000000,2.59029,0.00000,0.839589,0.000000,5.180581,0.000000,0.000000,2.59029,0.00000
talk.politics.misc,2.59029,0.000000,0.00000,2.59029,2.59029,0.00000,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
rec.sport.hockey,0.00000,0.000000,2.59029,0.00000,0.00000,2.59029,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,2.59029
sci.space,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,2.59029,0.000000,...,5.180581,0.00000,0.00000,0.000000,0.000000,0.000000,5.180581,0.000000,0.00000,0.00000


In [ ]:
tf.shape

(5, 39243)

In [ ]:
tf_icf_df

,0,1,2,3,4,5,6,7,8,9,...,44737,44738,44739,44740,44741,44742,44743,44744,44745,44746
0,0.00000,5.180581,0.00000,0.00000,0.00000,0.00000,0.000000,2.59029,0.00000,10.361162,...,0.000000,0.00000,2.59029,5.037532,82.889293,0.000000,0.000000,10.361162,0.00000,0.00000
1,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,7.770871,0.00000,0.00000,0.000000,...,0.000000,2.59029,0.00000,0.839589,0.000000,5.180581,0.000000,0.000000,2.59029,0.00000
2,2.59029,0.000000,0.00000,2.59029,2.59029,0.00000,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
3,0.00000,0.000000,2.59029,0.00000,0.00000,2.59029,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,2.59029
4,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,2.59029,0.000000,...,5.180581,0.00000,0.00000,0.000000,0.000000,0.000000,5.180581,0.000000,0.00000,0.00000


In [ ]:
x_train1 = pd.DataFrame(x_train)
x_train1

,_file_
242,xref cantaloupe srv cmu edu comp graphics comp...
201,xref cantaloupe srv cmu edu sci astro sci spac...
383,xref cantaloupe srv cmu edu talk abortion talk...
769,newsgroups rec sport hockey path cantaloupe sr...
546,path cantaloupe srv cmu edu crabapple srv cmu ...
...,...
302,xref cantaloupe srv cmu edu sci environment mi...
710,xref cantaloupe srv cmu edu sci environment mi...
145,xref cantaloupe srv cmu edu rec food cooking s...
48,path cantaloupe srv cmu edu crabapple srv cmu ...


In [ ]:
x_train1['_file_'] = x_train1['_file_'].apply(lambda x:x.split())
x_train1 = x_train1['_file_'].values

In [ ]:
x_train1

array([list(['newsgroups', 'comp', 'graphics', 'path', 'cantaloupe', 'srv', 'cmu', 'edu', 'das', 'news', 'harvard', 'edu', 'noc', 'near', 'net', 'nic', 'umass', 'edu', 'umassd', 'edu', 'ulowell', 'ulowell', 'edu', 'kseethar', 'kseethar', 'ulowell', 'edu', 'krishnan', 'seetharaman', 'subject', 'looking', 'info', 'quadratic', 'spline', 'bezier', 'conversion', 'message', 'bnna', 'ulowell', 'ulowell', 'edu', 'sender', 'usenet', 'ulowell', 'ulowell', 'edu', 'news', 'manager', 'ulowell', 'organization', 'umass', 'lowell', 'computer', 'science', 'date', 'sat', 'may', 'gmt', 'lines', 'looking', 'algorithm', 'pointers', 'papers', 'convert', 'quadratic', 'splines', 'cubic', 'splines', 'beizeirs', 'source', 'available', 'public', 'domain', 'please', 'let', 'know', 'thanks', 'much', 'krishnan', 'seetharaman', 'mail', 'kseethar', 'ulowell', 'edu', 'phone', 'snail', 'mail', 'department', 'computer', 'science', 'umass', 'lowell', 'lowell']),
       list(['xref', 'cantaloupe', 'srv', 'cmu', 'edu', 'ta

In [ ]:
x_test1 = pd.DataFrame(x_test)
x_test1['_file_'] = x_test1['_file_'].apply(lambda x:x.split())
x_test1 = x_test1['_file_'].values

In [ ]:
y_train1 = y_train.values

In [ ]:
y_test1 = y_test.values

### Building the matrix

In [ ]:
def matrix(x_train,y_train):
  arr = np.zeros((len(x_train),len(k_top_ele)))
  for i in range(len(x_train)):
    d = np.unique(x_train[i],return_counts=True)
    # arr=[]
    for j in range(len(k_top_ele)):
      
      if k_top_ele[j] in d[0]:
        # print(m[y_train[i]],j)
        # print(tf_icf[m[y_train[i]]][j])
        # arr[i][j] = tf_icf[m[y_train[i]]][int(kf[j])]
        ind = np.where(d[0] == k_top_ele[j])
        arr[i][j] = d[1][ind]
        

  return arr


mat = matrix(x_train1,y_train1)
mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 7., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
mat1 = matrix(x_test1,y_test1)
mat1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
mat1.shape

(999, 250)

In [ ]:
len(attributes)

39243

In [ ]:
tf_icf

array([[0.        , 3.21887582, 0.        , ..., 4.82831374, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.60943791,
        0.        ],
       [1.60943791, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.60943791, ..., 0.        , 0.        ,
        1.60943791],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

Extracting k features

In [ ]:
k_features=[]
#k=5#it should be user input
k=int(input("enter value"))
i=0
while(i<5):
  max=np.array(tf_icf[i]).argsort()[-k:][::-1]
  print(max)
  print(max)
  k_features.append(max)
  i+=1


In [ ]:
k_features

In [ ]:
kf = np.array([])

for i in range(len(k_features)):
  kf = np.append(kf,k_features[i])

kf

In [ ]:
kf = np.unique(kf)
kf

In [ ]:
len(kf)

250

In [ ]:
k_top_ele=[]
z=len(kf)
i=0
while(i<z):
  # for j in range(len(kf[i])):
  k_top_ele.append(attributes[int(kf[i])])
  i+=1


In [ ]:
k_top_ele=list(np.unique(k_top_ele))

In [ ]:
k_top_ele

### mapping to the label

In [ ]:
y_train2 = []
for i in range(len(y_train1)):
  y_train2.append(m[y_train1[i]])

y_test2 = []
for i in range(len(y_test1)):
  y_test2.append(m[y_test1[i]])

### segregating according to class

In [ ]:
dic={}
for i in range(len(mat)):
  if(dic.get(y_train2[i])==None):
    dic[y_train2[i]]=[]
  dic[y_train2[i]].append(mat[i])

### Finding count of terms in each class

In [ ]:
dic1={}
for i in range(5):
  temp = []
  for j in range(1,len(dic[i])):
    # print(temp)
    if(len(temp)==0):
      # print(dic[i][j-1])
      temp = np.add(dic[i][j],dic[i][j-1])
    else:
      temp = np.add(dic[i][j],temp)

  dic1[i] = temp




### Similarly for the test data

In [ ]:
dic2={}
for i in range(len(mat1)):
  if(dic2.get(y_test2[i])==None):
    dic2[y_test2[i]]=[]
  dic2[y_test2[i]].append(mat1[i])

In [ ]:
dic4={}
for i in range(len(mat1)):
  if(dic4.get(y_test2[i])==None):
    dic4[y_test2[i]]=[]
  dic4[y_test2[i]].append(x_test1[i])

In [ ]:
dic3={}
for i in range(5):
  temp = []
  for j in range(1,len(dic2[i])):
    # print(temp)
    if(len(temp)==0):
      # print(dic[i][j-1])
      temp = np.add(dic2[i][j],dic2[i][j-1])
    else:
      temp = np.add(dic2[i][j],temp)

  dic3[i] = temp

In [ ]:
dic1

### Naive Bayes Train

In [13]:

def train(dic,dic1,k_top_ele):
  prior = {} 
  cond_prob = {} 

  N = 0 
  for i in range(5):
      N += len(dic[i])
      
  for i in range(5):
      d = len(dic[i]) 
      
      
      if i not in prior:
          prior[i] = d/N 
      
    
      if i not in cond_prob:
          cond_prob[i] = {}
      
      summ = 0
      for j in range(len(dic1[i])):
          summ += dic1[i][j] 
      summ += len(k_top_ele) 
          
      for j in range(len(dic1[i])):
          tmp = dic1[i][j] + 1 
          cond_prob[i][j] = tmp/summ

  return prior,cond_prob

### Naive Bayes test

In [12]:
def test(dic2,dic4,prior,cond_prob,dic3,k_top_ele):
  acc_count = 0 

  N_test = 0 
  for i in range(5):
      N_test += len(dic2[i])

  ground = [] 
  predicted = [] 
      
  for i in range(5):
      test_docs = dic2[i]
      t2 = dic4[i]
      for s in range(len(test_docs)):
          
          
          vocab2 = set()
          
          

          for word in t2[s]:
              vocab2.add(word)
              
          
          score = {}
          
          max1 = -float("inf") 
          max1_class = 0 
          
          for c in range(5):
              if c not in score:
                  score[c] = 0
              
              score[c] += prior[c]
                  
              summ = 0
              for j in range(len(dic3[i])):
                summ += dic3[i][j] 
              summ += len(k_top_ele) #Smoothing (Denominator)

              for t in vocab2:
                  if t in k_top_ele:
                      # print(t)
                      # print(type(k_top_ele))
                      ind = k_top_ele.index(t)
                      # print(ind)
                      score[c] += cond_prob[c][ind]
                  else: #If the doc contains a term not in the training doc, we use smoothing to avoid -inf
                      tmp = (1/summ)*100
                      score[c] += tmp
                      
              if (score[c] > max1):
                  max1 = score[c]
                  max1_class = c
          
          predicted.append(max1_class)
          ground.append(i)
          
          if (max1_class == i): #Predicted class is same as actual class
              acc_count += 1
  mat2 = np.zeros((5, 5))

  for it in range(len(predicted)):
      mat2[int(ground[it])][int(predicted[it])] += 1

  df = pd.DataFrame(mat2)
  print("True vs Predicted")
  print(mat2)

  return acc_count,N_test

### Mapping to the labels

In [14]:
m = {
    "comp.graphics":0,
     "sci.med":1,
     "talk.politics.misc":2,
     "rec.sport.hockey":3,
     "sci.space":4
}

### Function to implement the naive bayes for different train test sizes

In [15]:
def size(t):

  from sklearn.model_selection import train_test_split
  y=df['label']
  x_train,x_test,y_train,y_test=train_test_split(df._file_,df.label,random_state=21,test_size=t)
  res=pd.concat([x_train,y_train],axis=1)
  
  df_fin=res.groupby('label')['_file_'].apply(list).reset_index(name="_file")
  li=[]
  for i in range(len(df_fin)) :
    flat = []
    j=0
    for sublist in df_fin.loc[i, "_file"]:
        # if j==0:
        #   print(sublist.split())
        # print(sublist)
        for w in sublist.split():
          flat.append(w)
        # for item in sublist:
        #     flat.append(item)
        j+=1
    li.append(flat)
  
  df_fin['terms'] = li
  df_fin.drop(['_file'],axis=1,inplace=True)
  temp_list=[]
  temp_list2=[]
  for i in range(len(df_fin)) :
    frq_list={}
    for word in df_fin.loc[i, "terms"]:
        if word not in frq_list.keys():
          frq_list[word]=1
        else:
          frq_list[word]+=1
    frq_list=dict( sorted(frq_list.items(), key=lambda x:(x[1], x[0]),reverse=True))
    temp_list2.append(frq_list)

  temp_list=df_fin['terms']
  df_fin['attri'] = temp_list
  df_fin['term_frq'] = temp_list2
  df_fin.index=df_fin["label"]
  labels=y.unique()

  attributes=[]
  for index,row in df_fin.iterrows():
    attributes.extend(row["attri"])
  attributes=np.unique(attributes)

  tf=pd.DataFrame(columns=attributes)
  for x in labels:
    for y in attributes:
      if y in df_fin.loc[x].term_frq.keys():
        tf.at[x,y]=df_fin.loc[x].term_frq[y]
  tf=tf.fillna(0)


  icf_list=[]
  for c in tf.columns:
    temp=0
    for index,row in tf.iterrows():
      if row[c]!=0:
        temp+=1
    icf_list.append(math.log(len(labels)/temp))

  i=0
  for j in tf.columns:
    tf[j]=tf[j]*icf_list[i]
    i=i+1

  tf_icf=tf.to_numpy()
  tf_icf_df=pd.DataFrame(tf_icf)
  tf_icf_df.to_csv('file.csv')

  x_train1 = pd.DataFrame(x_train)

  x_train1['_file_'] = x_train1['_file_'].apply(lambda x:x.split())
  x_train1 = x_train1['_file_'].values

  x_test1 = pd.DataFrame(x_test)
  x_test1['_file_'] = x_test1['_file_'].apply(lambda x:x.split())
  x_test1 = x_test1['_file_'].values

  y_train1 = y_train.values
  y_test1 = y_test.values

  k_features=[]
  #k=5#it should be user input
  k=int(input("enter value"))
  i=0
  while(i<5):
    max=np.array(tf_icf[i]).argsort()[-k:][::-1]
    # print(max)
    # print(max)
    k_features.append(max)
    i+=1

  kf = np.array([])

  for i in range(len(k_features)):
    kf = np.append(kf,k_features[i])

  kf = np.unique(kf)

  k_top_ele=[]
  z=len(kf)
  i=0
  while(i<z):
    # for j in range(len(kf[i])):
    k_top_ele.append(attributes[int(kf[i])])
    i+=1



  def matrix(x_train,y_train):
    arr = np.zeros((len(x_train),len(k_top_ele)))
    for i in range(len(x_train)):
      d = np.unique(x_train[i],return_counts=True)
      # arr=[]
      for j in range(len(k_top_ele)):
        
        if k_top_ele[j] in d[0]:
          # print(m[y_train[i]],j)
          # print(tf_icf[m[y_train[i]]][j])
          # arr[i][j] = tf_icf[m[y_train[i]]][int(kf[j])]
          ind = np.where(d[0] == k_top_ele[j])
          arr[i][j] = d[1][ind]

    return arr


  mat = matrix(x_train1,y_train1)


  mat1 = matrix(x_test1,y_test1)



  y_train2 = []
  for i in range(len(y_train1)):
    y_train2.append(m[y_train1[i]])

  y_test2 = []
  for i in range(len(y_test1)):
    y_test2.append(m[y_test1[i]])

  dic={}
  for i in range(len(mat)):
    if(dic.get(y_train2[i])==None):
      dic[y_train2[i]]=[]
    dic[y_train2[i]].append(mat[i])

  dic1={}
  for i in range(5):
    temp = []
    for j in range(1,len(dic[i])):
      # print(temp)
      if(len(temp)==0):
        # print(dic[i][j-1])
        temp = np.add(dic[i][j],dic[i][j-1])
      else:
        temp = np.add(dic[i][j],temp)

    dic1[i] = temp

  dic2={}
  for i in range(len(mat1)):
    if(dic2.get(y_test2[i])==None):
      dic2[y_test2[i]]=[]
    dic2[y_test2[i]].append(mat1[i])


  dic4={}
  for i in range(len(mat1)):
    if(dic4.get(y_test2[i])==None):
      dic4[y_test2[i]]=[]
    dic4[y_test2[i]].append(x_test1[i])

  dic3={}
  for i in range(5):
    temp = []
    for j in range(1,len(dic2[i])):
      # print(temp)
      if(len(temp)==0):
        # print(dic[i][j-1])
        temp = np.add(dic2[i][j],dic2[i][j-1])
      else:
        temp = np.add(dic2[i][j],temp)

    dic3[i] = temp

  prior,cond_prob = train(dic,dic1,k_top_ele)

  acc_count,N_test = test(dic2,dic4,prior,cond_prob,dic3,k_top_ele)
  print("Accuracy:", (acc_count/N_test)*100, "%")
  

In [16]:
size(0.5)

enter value400
True vs Predicted
[[427.   1.  22.   1.  32.]
 [  4. 325.   0.   0. 184.]
 [  6.   4. 462.   0.  21.]
 [  0.   0.   0. 522.   0.]
 [  1.   1.   0.   0. 485.]]
Accuracy: 88.91112890312249 %


In [ ]:
size(0.3)

enter value400
True vs Predicted
[[214.   1.   2.   1.  76.]
 [  0. 182.   0.   0. 124.]
 [  4.   1. 262.   2.  33.]
 [  0.   0.   0. 328.   0.]
 [  1.   1.   0.   0. 267.]]
Accuracy: 83.58905937291527 %


In [ ]:
size(0.2)

enter value400
True vs Predicted
[[155.   0.   1.   1.  28.]
 [  0. 127.   0.   0.  74.]
 [  3.   2. 174.   1.  29.]
 [  0.   0.   0. 228.   0.]
 [  0.   1.   0.   0. 175.]]
Accuracy: 85.98598598598599 %
